In [1]:
!pip install tensorflow
!pip install transformers
!pip install torch
!pip install pydot
!pip install graphviz

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 41.2 

In [2]:
import torch

import transformers
import tensorflow as tf
from tensorflow import keras
from transformers import AutoTokenizer, TFAutoModel
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics
import matplotlib.pyplot as plt

import boto3

In [3]:
#Get the file from s3
file = !aws s3 ls s3://purdue-ip-2023-readability/sil2-pred/ --recursive | sort | tail -n 1 | awk '{print $4}'
#print(file[0])

bucket='purdue-ip-2023-readability' # Or whatever you called your bucket
data_key = file[0] # Where the file is within your bucket
data_location = 's3://{}/{}'.format(bucket, data_key)
data = pd.read_csv(data_location)

sil2-pred/sample.csv


In [4]:
# Load LaBSE tokenizer and model
labse_model = TFAutoModel.from_pretrained("sentence-transformers/LaBSE")
labse_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")

#Defining max sentence length
sentence_length=64

2023-03-29 16:55:17.649036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 16:55:18.955271: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2023-03-29 16:56:12.098234: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at sentence-transformers/LaBSE.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [5]:
#Reading the input file as pandas dataframe
colnames = ['texts']
data = pd.read_csv(data_location, names=colnames, header=None)

In [6]:
#Define create embedding function
def embedding(dataset):
    e1 = []
    dataset=dataset
        
    #labels=np.array(dataset.complexity.values,dtype=np.float16)
    
    train_idx=np.array([*range(len(dataset))])
    texts=dataset.texts.values
    t1=texts[train_idx]
    for i in range(len(t1)):    

        input_ids = labse_tokenizer.encode(t1[i], add_special_tokens=True, return_tensors="tf",padding='max_length',max_length=sentence_length,truncation=True)
        outputs = labse_model(input_ids, output_hidden_states=True)
        sequence_output = outputs.hidden_states[-4:]    # getting last 4 layers
        embedding=np.array([np.array(sequence_output[j])[0] for j in range(len(sequence_output))])
        embedding=embedding.reshape((64,768,4))  #reshaping to fit the convolution layer
        e1.append(embedding)                     
    
    return e1

In [7]:
#Create and save embeddings
embeddings = embedding(data)
np.save('test1.npy',embeddings)
embeddings1 = np.load('test1.npy')

In [8]:
#Load the saved LSTM-CNN model
from keras.models import load_model
loaded_model = load_model('Model_96%_acc.h5')

val_pred = loaded_model.predict(embeddings1)
df = pd.DataFrame(val_pred)

2/2 [==============================] - 1s 73ms/step


In [9]:
df = df.rename(columns={0:'Complexity_Probability'})
df['Complexity'] = np.where(df['Complexity_Probability']<=0.5,'Simple','Complicated')
df['Complexity_Score'] = round(df['Complexity_Probability']*100,2)
df.drop('Complexity_Probability', axis=1, inplace=True)
df['Colorband'] = np.where(df['Complexity_Score']>=75,'Red',np.where(df['Complexity_Score']>=50,'Amber',np.where(df['Complexity_Score']>=25,'Yellow','Green')))

In [ ]:
#Saving the output file back to the s3 bucket
df1 = pd.concat([data,df], axis=1)
df1.to_csv('s3://purdue-ip-2023-readability/sil2-output/predictions.csv')